### Movie Recommender based on Plot summary & BERT

In [5]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained BERT model (paraphrase MiniLM model, a smaller model specificly for sentence/document embedding)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example sentences
sentence = "This is an example sentence. Second Sentence"

# Generate the sentences embedding
embedding = model.encode([sentence])

# Print the sentences embedding

print(embedding.shape)
print(embedding)

(1, 384)
[[ 2.25946620e-01  8.19291294e-01  2.67012775e-01  2.46059850e-01
   3.28651249e-01  2.98300564e-01  6.29707277e-01 -1.15905896e-01
  -1.49063960e-01 -1.42330453e-01  3.98270071e-01 -1.19548060e-01
   1.34256631e-01 -2.93876380e-02  7.61507154e-01  1.27342910e-01
   3.17410111e-01  3.82584840e-01 -5.28912485e-01 -1.35486409e-01
   6.93336606e-01 -9.44326669e-02 -1.91395566e-01 -3.87802660e-01
  -1.78278640e-01  2.22466439e-01 -3.98284718e-02 -8.84671360e-02
   9.32995826e-02  7.17376312e-03 -2.79101938e-01  6.25662059e-02
  -3.41902167e-01 -1.71655715e-01 -4.54989895e-02  1.39145091e-01
   1.76454216e-01  6.56842947e-01  9.97282118e-02 -3.96097824e-02
  -3.12061369e-01 -3.71141016e-01 -1.37536675e-01 -1.27933323e-01
  -4.48389113e-01 -3.43414307e-01  7.46563300e-02  7.34078661e-02
   5.99733219e-02 -5.02537012e-01 -3.26499760e-01  1.28363788e-01
  -5.48234165e-01  4.62552346e-02  5.11098020e-02 -9.08829551e-03
   1.07134797e-01  3.39609236e-01 -3.76013845e-01 -9.51930881e-02
 

In [6]:
import pandas as pd

df = pd.read_csv("data/tmdb_5000_movies.csv")

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can choose a different model if you prefer

# Generate sentence embeddings using SentenceTransformer
def get_sentence_embedding(text):
    return model.encode(text)

df = pd.read_csv("data/tmdb_5000_movies.csv")
df.dropna(subset=["overview"], inplace=True)  # Drop rows with missing overviews

df["embedding"] = df["overview"].apply(get_sentence_embedding)

# Recommendation function
def recommend_movies(movie_title, df, top_n=5):
    movie_row = df[df["title"].str.contains(movie_title, case=False, na=False)]
    if movie_row.empty:
        return f"Movie '{movie_title}' not found in dataset."
    
    movie_embedding = movie_row.iloc[0]["embedding"]

    # Compute cosine similarities
    similarities = []
    for index, row in df.iterrows():
        if row["title"] != movie_title:
            sim = cosine_similarity([movie_embedding], [row["embedding"]])[0][0]
            similarities.append((row["title"], sim))

    # Sort by similarity
    recommendations = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]
    return [rec[0] for rec in recommendations]


movie_title = "The Godfather"
recommended_movies = recommend_movies(movie_title, df)
print(f"Movies similar to '{movie_title}': {recommended_movies}")


Movies similar to 'The Godfather': ['The Godfather: Part III', 'The Godfather: Part II', 'Oldboy', 'Run All Night', 'Hostage']


Much cooler, right?! :))))